In [1]:
import pandas as pd
import numpy as np

main = pd.read_pickle("../Data/main_1.df")

In [23]:
# FEATURES TO CREATE:
# - Days since Last Race (continuous) * 
# - Jockey Win % (continuous) * 
# - Trainer Win % (continuous) * 
# - Horse Win % * 
# - Rating (continious) * 
# - Declared weight relative
# - Actual weight relative
# - Best record
# - Best distance time
# - Best going record
# - Best trainer record 
# - Start speed *
# - Win percent *
# - Last race result relative
# - Last race score relative *
# - Mean score relative * 

##### FIX DAYS SINCE LAST RACE ######
# avg = np.mean(main["days_since_last_race"])
# temp = np.zeros(len(main), dtype=int)
# for i in range(len(main)):
#     if main.iloc[i,:]["days_since_last_race"] == 0:
#         temp[i] = avg
#     else:
#         temp[i] = main.iloc[i,:]["days_since_last_race"]
# main["days_since_last_race"] = temp


##### MAKE LAST AND MEAN SCORES ######
horses = np.unique(main["horse_id"])
lastRaw = np.zeros(len(main))
lastRef = np.zeros(len(main))
meanRaw = np.zeros(len(main))
meanRef = np.zeros(len(main))
for i in range(len(horses)):
    temp = main[main["horse_id"]==horses[i]]
    tempI = temp.index
    
    for j in range(len(temp)):
        temp_2 = temp.iloc[j,:]
        date = temp_2["date"]
        
        beforeRaces = temp.loc[temp["date"]<date]
        
        if beforeRaces.shape[0] != 0:
            rawvals = [i for i in beforeRaces["raw_score"].values]
            refvals = [i for i in beforeRaces["refined_score"].values]
            lastRaw[tempI[j]] = rawvals[-1]
            lastRef[tempI[j]] = refvals[-1]
            meanRaw[tempI[j]] = np.mean(rawvals)
            meanRef[tempI[j]] = np.mean(refvals)

main["last_raw_score"] = lastRaw
main["last_refined_score"] = lastRef
main["mean_raw_score"] = meanRaw
main["mean_ref_score"] = meanRef

            
##### MAKE MEAN SCORES ######

In [49]:
# MAKE FEATURES RELATIVE TO THE HORSES IN THAT RACE
from sklearn.preprocessing import scale

rel_feat = [
    #"days_since_last_race", "horse_record", "jockey_record", "trainer_record", "raw_score", 
            #"refined_score", "horse_rating", "win_percent", "declared_weight", "actual_weight", 
            #"start_speed", "last_race_result", "last_refined_score", "last_raw_score", 
            "mean_raw_score", "mean_ref_score"]

for f in rel_feat:
    main[f+"_rel"] = 0.

races = np.unique(main["race_id"])
count = 0
for race_num in races:
    
    if count % 100 ==0:
        print("{}/{} races done".format(count, len(races)))
    count += 1
    
    race = main.loc[main["race_id"]==race_num]
    raceIndex = race.index
    
    for f in rel_feat:
        vals = race[f].to_numpy()
        if np.std(vals) < 0.00001:
            scaledVals = np.zeros(len(vals))
        else:
            scaledVals = scale(vals)
        
        for i in range(len(race)):
            main.iloc[raceIndex[i], main.columns.get_loc(f+"_rel")] = scaledVals[i]

0/6348 races done
100/6348 races done
200/6348 races done
300/6348 races done
400/6348 races done
500/6348 races done
600/6348 races done
700/6348 races done
800/6348 races done
900/6348 races done
1000/6348 races done
1100/6348 races done
1200/6348 races done
1300/6348 races done
1400/6348 races done
1500/6348 races done
1600/6348 races done
1700/6348 races done
1800/6348 races done
1900/6348 races done
2000/6348 races done
2100/6348 races done
2200/6348 races done
2300/6348 races done
2400/6348 races done
2500/6348 races done
2600/6348 races done
2700/6348 races done
2800/6348 races done
2900/6348 races done
3000/6348 races done
3100/6348 races done
3200/6348 races done
3300/6348 races done
3400/6348 races done
3500/6348 races done
3600/6348 races done
3700/6348 races done
3800/6348 races done
3900/6348 races done
4000/6348 races done
4100/6348 races done
4200/6348 races done
4300/6348 races done
4400/6348 races done
4500/6348 races done
4600/6348 races done
4700/6348 races done
4800

In [46]:
for i in main.columns:
    print(i)

race_id
horse_no
horse_id
result
won
lengths_behind
horse_age
horse_country
horse_type
horse_rating
horse_gear
declared_weight
actual_weight
draw
position_sec1
position_sec2
position_sec3
position_sec4
position_sec5
position_sec6
behind_sec1
behind_sec2
behind_sec3
behind_sec4
behind_sec5
behind_sec6
time1
time2
time3
time4
time5
time6
finish_time
win_odds
place_odds
trainer_id
jockey_id
race_size
date
venue
race_no
distance
going
config
horse_ratings
prize
race_class
place_combination1
place_combination2
place_combination3
place_combination4
last_race_result
win_percent
avg_distance_time
normal_avg_distance_time
going_type
going_type_record
actual_weight_scaled
declared_weight_scaled
horse_race_count
going_type_fast
going_type_slow
going_type_wet
jockey_record
trainer_record
horse_record
days_since_last_race
weight_change
weight_change_over_time
weight_change_from_average
weight_change_increase
normalized_result
venue_change
venue_record
best_odds
best_win_percent
best_distance_time
b

In [25]:


a = [1,2,3,1,2,10, 3, 5]
s = scale(a)
print(s)

[-0.85398649 -0.49441323 -0.13483997 -0.85398649 -0.49441323  2.38217285
 -0.13483997  0.58430655]


In [45]:
main = main.drop(["last_race_refined_score_rel", "last_race_raw_score_rel"], axis=1)

In [50]:
main.tail()

,race_id,horse_no,horse_id,result,won,lengths_behind,horse_age,horse_country,horse_type,horse_rating,...,horse_rating_rel,win_percent_rel,declared_weight_rel,actual_weight_rel,start_speed_rel,last_race_result_rel,last_refined_score_rel,last_raw_score_rel,mean_raw_score_rel,mean_ref_score_rel
79442,6348,10,1238,6,0,1.25,5,AUS,Gelding,87,...,-0.255118,-0.261409,0.884500,-0.629278,0.790108,-0.259213,0.337481,0.350176,0.283424,0.282435
79443,6348,11,985,7,0,2.25,5,NZ,Gelding,84,...,-0.969449,-0.450396,-0.903499,-1.306962,1.569702,0.157169,0.312354,0.325042,0.292913,0.284986
79444,6348,12,3147,10,0,3.00,5,NZ,Gelding,83,...,-1.207560,-0.429017,0.249095,-1.081067,-0.717636,-0.933356,0.271711,0.269366,0.266023,0.270420
79445,6348,13,1982,8,0,2.50,3,AUS,Gelding,82,...,-1.445670,3.162600,1.992763,-1.306962,1.310413,-1.151461,0.259764,0.257416,0.258591,0.258434
79446,6348,14,3491,11,0,4.25,3,IRE,Gelding,81,...,-1.683781,-0.728319,-0.578408,-1.532856,1.350889,1.683905,0.309028,0.306694,0.338893,0.331149


In [51]:
main.to_pickle("../Data/main_1.df")